# Magic Methods

* [The Internals of Operations](#The-Internals-of-Operations)
* [Overloading Built-in Functions](#Overloading-Built-in-Functions)
* [ Overloading Built-in Operators](#Overloading-Built-in-Operators)
* [Indexing and Slicing Your Objects](#Indexing-and-Slicing-Your-Objects)
* [Reverse Operators](#Reverse-Operators)

If you’ve used the + or * operator on a str object in Python, you must have noticed its different behavior when compared to int or float objects:

In [3]:
1 + 2

3

In [1]:
'rasool' + 'ahadi'

'rasoolahadi'

You might have wondered how the same built-in operator or function shows different behavior for objects of different classes. This is called operator overloading or function overloading respectively. This article will help you understand this mechanism, so that you can do the same in your own Python classes and make your objects more Pythonic.

> **Fun fact:** Due to the naming convention used for these methods, they are also called **dunder** methods which is a shorthand for double underscore methods. Sometimes they’re also referred to as special methods or magic methods. We prefer dunder methods though!

<hr/>

## The Internals of Operations

Every class in Python defines its own behavior for built-in functions and methods. When you pass an instance of some class to a built-in function or use an operator on the instance, it is actually equivalent to calling a special method with relevant arguments.

In [15]:
a = ["rasool", "amir"]
len(a)

2

In [17]:
a.__len__()

2

In [19]:
dir(a)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

## Overloading Built-in Functions

#### Giving a Length to Your Objects Using len()

In [23]:
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer
    def __len__(self):
        return len(self.cart)

order = Order(['banana', 'apple', 'mango'], 'rasool')
len(order)

3

As you can see, you can now use ``len()`` to directly obtain the length of the cart. 

> **note:** . When you don’t have the ``__len__()`` method defined but still call len() on your object, you get a TypeError:

But, when **overloading** len(), you should keep in mind that Python requires the function to return an integer.

In [27]:
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer
    def __len__(self):
        return float(len(self.cart))
order = Order(['banana', 'apple', 'mango'], 'Real Python')
len(order)

TypeError: 'float' object cannot be interpreted as an integer

> **Note:** len should be int

#### Making Your Objects Work With abs()

There are no restrictions on the return value of abs(), and you get a TypeError when the special method is absent in your class definition.

In [6]:
class Vector:
    def __init__(self, x_comp, y_comp):
        self.x_comp = x_comp
        self.y_comp = y_comp
    def __abs__(self):
        return (self.x_comp ** 2 + self.y_comp ** 2) ** 0.5
        
vector = Vector(3, 4)
abs(vector)

5.0

#### Printing Your Objects Prettily Using str()

 You can define the string format your object should be displayed in when passed to str() by defining the ``__str__()`` method in your class. Moreover, ``__str__()`` is the method that is used by Python when you call ``print()`` on your object.

In [14]:
class Vector:
    def __init__(self, x_comp, y_comp):
        self.x_comp = x_comp
        self.y_comp = y_comp
    def __abs__(self):
        return (self.x_comp ** 2 + self.y_comp ** 2) ** 0.5
    def __str__(self):    
         return f'{self.x_comp}i{self.y_comp:+}j'
vector = Vector(3, -4)
print(vector)


3i-4j


> **note:** The ``repr()`` built-in is used to obtain the parsable string representation of an object. 

In [16]:
vector

#### Making Your Objects Truthy or Falsey Using bool()

In [18]:
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer
    def __bool__(self):
        return len(self.cart) > 0
order1 = Order(['banana', 'apple', 'mango'], 'Real Python')
order2 = Order([], 'Python')

bool(order1)


True

In [19]:
bool(order2)

False

> **Note:** When the ``__bool__()`` special method is not implemented in a class, the value returned by ``__len__()`` is used as the truth value, where a non-zero value indicates True and a zero value indicates False.

## Overloading Built-in Operators

Changing the behavior of operators is just as simple as changing the behavior of functions. You define their corresponding special methods in your class, and the operators work according to the behavior defined in these methods.

#### Making Your Objects Capable of Being Added Using +

The special method corresponding to the + operator is the ``__add__()`` method. Adding a custom definition of ``__add__()`` changes the behavior of the operator. 

> **note:** It is recommended that ``__add__()`` returns a new instance of the class instead of modifying the calling instance itself.

In [26]:
a = "rasool"
a

'rasool'

In [27]:
a + "ahadi"
a

'rasool'

In [28]:
a = a +"ahadi"
a

'rasoolahadi'

Let’s implement the ability to append new items to our cart in the Order class using the operator. We’ll follow the recommended practice and make the operator return a new Order instance that has our required changes instead of making the changes directly to our instance:

In [3]:
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer
    def __add__(self, other):
        new_cart = self.cart.copy()
        new_cart.append(other)
        return Order(new_cart, self.customer)
order = Order(['banana', 'apple'], 'rasool ahadi')
order = order+"orange"
order.cart


['banana', 'apple', 'orange']

Similarly, you have the ``__sub__()``, ``__mul__()``, ``__div__()`` and other special methods which define the behavior of -, *, and so on.

#### Shortcuts:the += Operator

The += operator stands as a shortcut to the expression obj1 = obj1 + obj2. The special method corresponding to it is ``__iadd__()``. The ``__iadd__()`` method should make changes directly to the self argument and return the result, which may or may not be self.

> **remember** : Roughly, any += use on two objects is equivalent to this:
```python
result = obj1 + obj2
obj1 = result
```

In [7]:
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer
    def __add__(self, other):
        new_cart = self.cart.copy()
        new_cart.append(other)
        return Order(new_cart, self.customer)
    def __iadd__(self, other):
        self.cart.append(other)
        return self
order = Order(['banana', 'apple'], 'rasool ahadi')
order += "orange"
order.cart


['banana', 'apple', 'orange']

> **note:** As can be seen, any change is made directly to self and it is then **return self**.

> Similar to ``__iadd__()``, you have ``__isub__()``, ``__imul__()``, ``__idiv__()`` and other special methods which define the behavior of -=, *=, /=, and others alike.

## Indexing and Slicing Your Objects

The [] operator is called the indexing operator and is used in various contexts in Python such as getting the value at an index in sequences, getting the value associated with a key in dictionaries, or obtaining a part of a sequence through slicing. You can change its behavior using the ``__getitem__()`` special method.

In [13]:
class Order:
    def __init__(self, cart, customer):
        self.cart = list(cart)
        self.customer = customer
    def __getitem__(self, key):
        return self.cart[key]

order = Order(['banana', 'apple'], 'rasool ahadi')
order[0]

'banana'

> **Note:** As long as you’re using data structures that support slicing (lists, tuples, strings, and so on), you can configure your objects to directly slice the structure:

## Reverse Operators

While defining the ``__add__()``, ``__sub__()``, ``__mul__()``, and similar special methods allows you to use the operators when your class instance is the left-hand side operand, the operator will not work if the class instance is the right-hand side operand

If your class represents a mathematical entity like a vector, a coordinate, or a complex number, applying the operators should work in both the cases since it is a valid mathematical operation.

Moreover, if the operators work only when the instance is the left operand, we are violating the fundamental principle of commutativity in many cases. Therefore, to help you make your classes mathematically correct, Python provides you with reverse special methods such as ``__radd__()``, ``__rsub__()``, ``__rmul__()``, and so on.